In [1]:
!pip install lifetimes
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter
from lifetimes.plotting import plot_period_transactions

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
pd.set_option('display.float_format', lambda x: '%.4f' % x)
from sklearn.preprocessing import MinMaxScaler

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.2/584.2 kB 804.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.8/48.8 kB 2.6 MB/s eta 0:00:00


In [2]:
df_ = pd.read_csv('../input/flo-rfm/flo_data_20k.csv')
df = df_.copy()
df.head()

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12
0,cc294636-19f0-11eb-8d74-000d3a38a36f,Android App,Offline,2020-10-30,2021-02-26,2021-02-21,2021-02-26,4.0000,1.0000,139.9900,799.3800,[KADIN]
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,Android App,Mobile,2017-02-08,2021-02-16,2021-02-16,2020-01-10,19.0000,2.0000,159.9700,1853.5800,"[ERKEK, COCUK, KADIN, AKTIFSPOR]"
2,69b69676-1a40-11ea-941b-000d3a38a36f,Android App,Android App,2019-11-27,2020-11-27,2020-11-27,2019-12-01,3.0000,2.0000,189.9700,395.3500,"[ERKEK, KADIN]"
3,1854e56c-491f-11eb-806e-000d3a38a36f,Android App,Android App,2021-01-06,2021-01-17,2021-01-17,2021-01-06,1.0000,1.0000,39.9900,81.9800,"[AKTIFCOCUK, COCUK]"
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,Desktop,Desktop,2019-08-03,2021-03-07,2021-03-07,2019-08-03,1.0000,1.0000,49.9900,159.9900,[AKTIFSPOR]


# Data Preparation

In [3]:
df.describe()

,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online
count,19945.0000,19945.0000,19945.0000,19945.0000
mean,3.1109,1.9139,253.9226,497.3217
std,4.2256,2.0629,301.5329,832.6019
min,1.0000,1.0000,10.0000,12.9900
25%,1.0000,1.0000,99.9900,149.9800
50%,2.0000,1.0000,179.9800,286.4600
75%,4.0000,2.0000,319.9700,578.4400
max,200.0000,109.0000,18119.1400,45220.1300


In [4]:
def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit


def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit


In [5]:
outlier_cols = ["order_num_total_ever_online", "order_num_total_ever_offline", \
                "customer_value_total_ever_offline","customer_value_total_ever_online"]

for col in outlier_cols:
    replace_with_thresholds(df,col)

In [6]:
df.describe()

,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online
count,19945.0000,19945.0000,19945.0000,19945.0000
mean,3.0923,1.8862,251.9212,489.7054
std,3.8137,1.4347,251.0226,632.6064
min,1.0000,1.0000,10.0000,12.9900
25%,1.0000,1.0000,99.9900,149.9800
50%,2.0000,1.0000,179.9800,286.4600
75%,4.0000,2.0000,319.9700,578.4400
max,48.5000,16.0000,3019.8820,7799.5410


In [7]:
# Creating total_order and total_value columns to keep sum of the online and offline sales

df['total_order'] = (df.order_num_total_ever_online + df.order_num_total_ever_offline)

df['total_value'] = (df.customer_value_total_ever_online + df.customer_value_total_ever_offline)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19945 entries, 0 to 19944
Data columns (total 14 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   master_id                          19945 non-null  object 
 1   order_channel                      19945 non-null  object 
 2   last_order_channel                 19945 non-null  object 
 3   first_order_date                   19945 non-null  object 
 4   last_order_date                    19945 non-null  object 
 5   last_order_date_online             19945 non-null  object 
 6   last_order_date_offline            19945 non-null  object 
 7   order_num_total_ever_online        19945 non-null  float64
 8   order_num_total_ever_offline       19945 non-null  float64
 9   customer_value_total_ever_offline  19945 non-null  float64
 10  customer_value_total_ever_online   19945 non-null  float64
 11  interested_in_categories_12        19945 non-null  obj

In [9]:
# Changing the data type of the date columns to datetime

for col in df.columns:
    if 'date' in col:
        df[col] = pd.to_datetime(df[col])

In [10]:
df.dtypes

master_id                                    object
order_channel                                object
last_order_channel                           object
first_order_date                     datetime64[ns]
last_order_date                      datetime64[ns]
last_order_date_online               datetime64[ns]
last_order_date_offline              datetime64[ns]
order_num_total_ever_online                 float64
order_num_total_ever_offline                float64
customer_value_total_ever_offline           float64
customer_value_total_ever_online            float64
interested_in_categories_12                  object
total_order                                 float64
total_value                                 float64
dtype: object

# CLTV Data Structure

In [11]:
#setting an analysis date for more readable recency column

analysis_date = df.last_order_date.max() + dt.timedelta(days=2)
analysis_date

Timestamp('2021-06-01 00:00:00')

In [12]:
recency = df.apply(lambda x : ((x.last_order_date - x.first_order_date).days) / 7,axis=1)

monetary = df.apply(lambda x : x.total_value / x.total_order,axis=1)

cltv = df.agg({'master_id' : lambda x: x,
               'first_order_date': lambda date: ((analysis_date - date).days) / 7,
               'total_order' : lambda x: x
              })

In [13]:
cltv = pd.concat([cltv,recency,monetary],axis=1)
cltv.columns = ['customer_id','T_weekly','frequency','recency_cltv_weekly','monetary_cltv_avg']

In [14]:
cltv.head()

,customer_id,T_weekly,frequency,recency_cltv_weekly,monetary_cltv_avg
0,cc294636-19f0-11eb-8d74-000d3a38a36f,30.5714,5.0000,17.0000,187.8740
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,224.8571,21.0000,209.8571,95.8833
2,69b69676-1a40-11ea-941b-000d3a38a36f,78.8571,5.0000,52.2857,117.0640
3,1854e56c-491f-11eb-806e-000d3a38a36f,20.8571,2.0000,1.5714,60.9850
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,95.4286,2.0000,83.1429,104.9900


### Fitting BG/NBD model

In [15]:
bgf = BetaGeoFitter(penalizer_coef=0.001)

bgf.fit(cltv['frequency'],
        cltv['recency_cltv_weekly'],
        cltv['T_weekly'])

<lifetimes.BetaGeoFitter: fitted with 19945 subjects, a: 0.00, alpha: 76.17, b: 0.00, r: 3.66>

### Expected Number of Puchases with BG/NBD 

In [16]:
cltv['exp_sales_3_month'] = bgf.predict(12,
            cltv['frequency'],
            cltv['recency_cltv_weekly'],
            cltv['T_weekly'])

In [17]:
cltv['exp_sales_6_month'] = bgf.predict(24,
            cltv['frequency'],
            cltv['recency_cltv_weekly'],
            cltv['T_weekly'])

### Fitting Gamma-Gamma Model

In [18]:
ggf = GammaGammaFitter(penalizer_coef=0.01)

ggf.fit(cltv['frequency'].astype(int), cltv['monetary_cltv_avg'])

<lifetimes.GammaGammaFitter: fitted with 19945 subjects, p: 4.15, q: 0.47, v: 4.08>

### Expected Average Profit with Gamma-Gamma

In [19]:
cltv['expected_average_profit'] = ggf.conditional_expected_average_profit(cltv['frequency'],
                                                                          cltv['monetary_cltv_avg'])

In [20]:
cltv.head()

,customer_id,T_weekly,frequency,recency_cltv_weekly,monetary_cltv_avg,exp_sales_3_month,exp_sales_6_month,expected_average_profit
0,cc294636-19f0-11eb-8d74-000d3a38a36f,30.5714,5.0000,17.0000,187.8740,0.9739,1.9479,193.6327
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,224.8571,21.0000,209.8571,95.8833,0.9832,1.9663,96.6650
2,69b69676-1a40-11ea-941b-000d3a38a36f,78.8571,5.0000,52.2857,117.0640,0.6706,1.3412,120.9676
3,1854e56c-491f-11eb-806e-000d3a38a36f,20.8571,2.0000,1.5714,60.9850,0.7004,1.4008,67.3201
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,95.4286,2.0000,83.1429,104.9900,0.3960,0.7921,114.3251


### CLTV Values for 6 Month

In [21]:
cltv['cltv'] = cltv.exp_sales_6_month * cltv.expected_average_profit

In [22]:
# Top 20 customer with highest CLTV value
cltv.sort_values(by='cltv',ascending=False).head(20)

,customer_id,T_weekly,frequency,recency_cltv_weekly,monetary_cltv_avg,exp_sales_3_month,exp_sales_6_month,expected_average_profit,cltv
9055,47a642fe-975b-11eb-8c2a-000d3a38a36f,7.8571,4.0000,2.8571,1401.7705,1.0944,2.1888,1449.0298,3171.5923
13880,7137a5c0-7aad-11ea-8f20-000d3a38a36f,13.1429,11.0000,6.1429,758.0437,1.9701,3.9402,767.3184,3023.3998
17323,f59053e2-a503-11e9-a2fc-000d3a38a36f,101.0000,7.0000,51.7143,1106.4671,0.7222,1.4445,1127.6115,1628.8076
12438,625f40a2-5bd2-11ea-98b0-000d3a38a36f,74.5714,16.0000,74.2857,501.8451,1.5653,3.1306,506.1377,1584.5239
7330,a4d534a2-5b1b-11eb-8dbd-000d3a38a36f,67.2857,52.5000,62.7143,164.6328,4.6980,9.3959,165.1119,1551.3788
8868,9ce6e520-89b0-11ea-a6e7-000d3a38a36f,34.4286,8.0000,3.4286,601.2262,1.2655,2.5309,611.4926,1547.6338
6402,851de3b4-8f0c-11eb-8cb8-000d3a38a36f,9.4286,2.0000,8.2857,862.6900,0.7939,1.5878,923.6797,1466.6624
6666,53fe00d4-7b7a-11eb-960b-000d3a38a36f,13.0000,17.0000,9.7143,259.8653,2.7807,5.5614,262.0729,1457.4865
19538,55d54d9e-8ac7-11ea-8ec0-000d3a38a36f,58.7143,31.0000,52.5714,228.5300,3.0838,6.1676,229.6069,1416.1141
14858,031b2954-6d28-11eb-99c4-000d3a38a36f,15.5714,3.0000,14.8571,743.5867,0.8716,1.7431,778.0503,1356.2416


# Segmentation

In [23]:
cltv['segment'] = pd.qcut(cltv.cltv,4,list('DBCA'))

In [24]:
cltv

,customer_id,T_weekly,frequency,recency_cltv_weekly,monetary_cltv_avg,exp_sales_3_month,exp_sales_6_month,expected_average_profit,cltv,segment
0,cc294636-19f0-11eb-8d74-000d3a38a36f,30.5714,5.0000,17.0000,187.8740,0.9739,1.9479,193.6327,377.1680,A
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,224.8571,21.0000,209.8571,95.8833,0.9832,1.9663,96.6650,190.0747,C
2,69b69676-1a40-11ea-941b-000d3a38a36f,78.8571,5.0000,52.2857,117.0640,0.6706,1.3412,120.9676,162.2384,C
3,1854e56c-491f-11eb-806e-000d3a38a36f,20.8571,2.0000,1.5714,60.9850,0.7004,1.4008,67.3201,94.3036,D
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,95.4286,2.0000,83.1429,104.9900,0.3960,0.7921,114.3251,90.5543,D
...,...,...,...,...,...,...,...,...,...,...
19940,727e2b6e-ddd4-11e9-a848-000d3a38a36f,88.4286,3.0000,41.1429,133.9867,0.4858,0.9716,141.3604,137.3414,B
19941,25cd53d4-61bf-11ea-8dd8-000d3a38a36f,65.2857,2.0000,42.2857,195.2350,0.4804,0.9609,210.7224,202.4744,C
19942,8aea4c2a-d6fc-11e9-93bc-000d3a38a36f,89.8571,3.0000,88.7143,210.9800,0.4816,0.9632,221.7752,213.6160,C
19943,e50bb46c-ff30-11e9-a5e8-000d3a38a36f,113.8571,6.0000,98.4286,168.2950,0.6102,1.2204,172.6474,210.7073,C
